In [9]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.operators.sampling.lhs import LHS
from pymoo.optimize import minimize

In [10]:
class BMS:

    def __init__(self, sim_time = 1000, gamma=0.5, theta = 1):
      self.sim_time = sim_time      
      self.gamma = gamma
      self.theta = theta

    def simulate(self, i_ext):
      v = [] # Vector de voltaje de membrana
      spike_train = [] # Tren de pulsos

      v.append(0.) # Inicializamos el vector de ... con 0

      # Ciclo para obtener el tren de pulsos
      for k in range(1, self.sim_time):
          # Operación para obtener el valor de ...
          _v = self.gamma * v[-1] * (1 - (1 if v[-1] >= self.theta else 0)) + i_ext

          # Comprobar si la neurona descanso
          if (_v >= self.theta):
              spike_train.append(k)
              _v = self.theta

          v.append(_v)

      return spike_train, len(spike_train) / self.sim_time   


In [11]:
class BMS_Training(Problem):
    
    def __init__(self, Xtr, ytr, sim_time, gamma, theta, n_var, xl, xu):
        super().__init__(n_var=n_var, xl=xl, xu=xu)
        self.Xtr = Xtr
        self.ytr = ytr
        self.neuron = BMS(sim_time, gamma, theta)
        self.n_class = len(np.unique(ytr))

    def _evaluate(self, x, out, *args, **kwargs):
      fit = np.zeros(len(x))
      firing_rates = {i:[] for i in range(self.n_class)}      
      for i, w in enumerate(x):

        # Ciclo para obtener las tasas de disparo por cada clase
        for _x, _y in zip(self.Xtr, self.ytr):
          _iext = _x @ w # Generar el estímulo a partir del patron actual y los pesos
          _,_fr = self.neuron.simulate(_iext) # Obtener la cantidad de disparos que hizo el patrón actual
          firing_rates[_y].append(_fr) # Agregar la tasa de disparo del patrón a la clase correspondiente
      
        _m = np.zeros(self.n_class)
        _sd = np.zeros(self.n_class)

        for _k in firing_rates.keys():
          _m[_k] = np.mean(firing_rates[_k])
          _sd[_k] = np.std(firing_rates[_k])

        sum_dist = 0
        for j in range(len(_m)):
          for k in range(j + 1, len(_m)):
            tmp = _m[j] - _m[k]
            sum_dist += np.sqrt(np.dot(tmp, tmp))

        sum_dist = 1. / sum_dist if sum_dist > 0. else 1000000
        sum_sd = np.sum(_sd) if np.sum(_sd) > 0 else 1000000
        fit[i] = sum_dist + sum_sd
              
      out['F'] = fit  

'''
if __name__ == "__main__":
    X, y =  load_iris(return_X_y=True)
    Xtr,Xte,ytr,yte = train_test_split(X, y, test_size=0.1, stratify=y)
    problem = BMS_Training(Xtr, ytr, n_var=X.shape[1], xl=np.repeat(-1,X.shape[1]), xu=np.repeat(1,X.shape[1]))
    
    algorithm = PSO(pop_size=30)

    solution = minimize(
        problem = problem,
        algorithm = algorithm,
        termination = ('n_gen', 10),
        verbose = True
    )
    print(solution.X)
'''
    

'\nif __name__ == "__main__":\n    X, y =  load_iris(return_X_y=True)\n    Xtr,Xte,ytr,yte = train_test_split(X, y, test_size=0.1, stratify=y)\n    problem = BMS_Training(Xtr, ytr, n_var=X.shape[1], xl=np.repeat(-1,X.shape[1]), xu=np.repeat(1,X.shape[1]))\n    \n    algorithm = PSO(pop_size=30)\n\n    solution = minimize(\n        problem = problem,\n        algorithm = algorithm,\n        termination = (\'n_gen\', 10),\n        verbose = True\n    )\n    print(solution.X)\n'

In [12]:
class BMS_Classifier(BaseEstimator):
    
    def __init__(self, sim_time=100, gamma=0.5, theta=1):
        self.sim_time = sim_time
        self.gamma = gamma
        self.theta = theta        
        self.neuron = BMS(self.sim_time, self.theta, self.gamma)
        self._m = None
        self.n_class = None

    def fit(self, X, y):
      problem = BMS_Training(Xtr, ytr, self.sim_time, self.gamma, self.theta, n_var=X.shape[1], xl=np.repeat(-1,X.shape[1]), xu=np.repeat(1,X.shape[1]))
    
      algorithm = DE(
          sampling=LHS(),
          variant="DE/rand/1/bin",
          pop_size=30,
          CR=0.3,
          dither="vector",
          jitter=False)

      solution = minimize(
          problem = problem,
          algorithm = algorithm,
          termination = ('n_gen', 10),
          #verbose = True
      )
      self.n_class = len(np.unique(ytr))
      self.w = solution.X

      firing_rates = {i:[] for i in range(len(np.unique(ytr)))}
      for _x, _y in zip(Xtr, ytr):
          _iext = _x @ self.w # Generar el estímulo a partir del patron actual y los pesos
          _,_fr = self.neuron.simulate(_iext) # Obtener la cantidad de disparos que hizo el patrón actual
          firing_rates[_y].append(_fr) # Agregar la tasa de disparo del patrón a la clase correspondiente
      self._m = np.zeros(len(np.unique(ytr)))

      for _k in firing_rates.keys():
        self._m[_k] = np.mean(firing_rates[_k])      


    def predict(self,X):
      
      pred = np.zeros(X.shape[0], dtype=int)
      for j, x in enumerate(X):
        _iext = x @ self.w 
        _,_fr = self.neuron.simulate(_iext)
        dif = np.zeros(self.n_class)
        
        for i, m in enumerate(self._m):                    
          dif[i] = np.abs(_fr-m)
        pred[j] = np.argmin(dif)

      return pred

if __name__ == "__main__":
  X, y =  load_iris(return_X_y=True)

  X_tr,X_te,y_tr,y_te = train_test_split(X, y, test_size=0.1, stratify=y)

  bmsC = BMS_Classifier(gamma=0.7)
  bmsC.fit(X_tr, y_tr)
  bmsC_pred = bmsC.predict(X_te)
  print(f"Accuracy Classifier: {round(accuracy_score(bmsC_pred, y_te), 2)}") 
  
  bmsE = BaggingClassifier(base_estimator=bmsC, n_estimators=10, n_jobs=-1)
  bmsE.fit(X_tr, y_tr)
  bmsE_pred = bmsE.predict(X_te)
  print(f"Accuracy Bagging: {round(accuracy_score(bmsE_pred, y_te), 2)}")

Accuracy Classifier: 0.67
Accuracy Bagging: 0.67
